In [ ]:
import numpy as np
import xarray as xr
from modules.integrator import forward_euler_final
from modules.classes import Quantity2D, Analytic, Interpolate
from modules.tests import plot_mass_conservation, test_gaussian
import matplotlib.pyplot as plt

# Initialize a constant diffusion coefficient function
diffusion = Analytic(lambda x, y: x*y)
diffusion.set_partial_x(lambda x, y: y)
diffusion.set_partial_y(lambda x, y: x)

In [ ]:
n_grid = 100
n_time = 500
dt = 0.0001
dx = 0.01
dy = 0.01

conc = Quantity2D(
    n_grid,
    n_time,
    (-0.5*n_grid*dx, 0.5*n_grid*dx),
    (-0.5*n_grid*dy, 0.5*n_grid*dy),
    (0, dt*n_time),
)

xcoords = conc.xcoords
ycoords = conc.ycoords
X, Y = np.meshgrid(xcoords, ycoords)
initial_condition = 1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)
sources =  1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)
#initial_condition = np.empty_like(X)
#initial_condition[n_grid//2, n_grid//2] = 100
diffarr = 0.02*np.ones((10, 10))
diffarr[:,4:-4] = 0.2 # Vary along x; partial_x should be constant
xint = np.linspace(-0.5*n_grid*dx, 0.5*n_grid*dx, 10)
yint = np.linspace(-0.5*n_grid*dy, 0.5*n_grid*dy, 10)
diffusion = Interpolate(diffarr, xint, yint, s=0)
diffarr = 0.02*np.ones((10, 10))
diffarr[:,4:-4] = 0.2 # Vary along x; partial_x should be constant
diffusion.plot_2D(func='func')



In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, initial_condition)

In [ ]:
result_ds = forward_euler_final(conc, diffusion, initial_condition, sources)
result_ds

In [ ]:
xr.plot.contourf(result_ds['concentration'].isel(t=100), levels=50)


In [ ]:
fig, ax = plot_mass_conservation(result_ds)

In [ ]:
diff = test_gaussian(forward_euler_final)

In [ ]:
xr.plot.contourf(diff.isel(t=100), levels=30)

In [ ]:
diff.median(dim=('x','y')).plot()

In [ ]:
diff.mean(dim=('x','y')).plot()

In [ ]:
xrange = (-10, 10)
yrange = (-10, 10)
trange=(0, 1)
n_grid = 100
n_time = 500
conc = Quantity2D(
    n_grid,
    n_time,
    xrange,
    yrange,
    trange,
)
xcoords = conc.xcoords
ycoords = conc.ycoords
tcoords = conc.tcoords
X, Y = np.meshgrid(xcoords, ycoords, indexing='ij')
initial_condition =  (1/(4*np.pi))*np.exp(- (X**2 + Y**2)/1)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, initial_condition)